### Fix the setup

##### Open Terminal, it is below the cpu logo on the right panel in lightning ai

##### Install ollama

```curl -fsSL https://ollama.com/install.sh | sh```

##### Pull ollama model, llama 3.2 or qwen2.5-coder

```ollama pull qwen2.5-coder:7b```

##### Download uv

```curl -LsSf https://astral.sh/uv/install.sh | sh```

##### Create venv using uv

```! uv venv --python 3.10```

##### Activate the environment
``` source .venv/bin/activate```

##### Install the packages in the Terminal
```uv pip install jupyterlab langchain langchain_core langchain_ollama psutil speedtest-cli```

In [1]:
import shutil
import sys
from datetime import datetime
from zoneinfo import ZoneInfo
from langchain_core.messages import HumanMessage, ToolMessage
from langchain_core.tools import tool
from langchain_ollama import ChatOllama


messages = []

@tool(parse_docstring=True)
def get_disk_usage():
    """Retrieves disk usage. Call this whenever you need to know the disk usage, for example when a customer asks "What is the disk usage?"
    Args: None

    Returns:
        dict: A dictionary containing disk usage statistics with the following keys:
            - total (str): Total disk space in GB
            - used (str): Used disk space in GB
            - free (str): Free disk space in GB
    """
    path = "/"
    total, used, free = shutil.disk_usage(path)
    gb = 1024 * 1024 * 1024

    return {
        "total": f"{total / gb:.2f} GB",
        "used": f"{used / gb:.2f} GB",
        "free": f"{free / gb:.2f} GB",
    }

In [2]:
# You can implement one of the following options: 
import os
import psutil
import shutil
import socket
import platform
import subprocess
from datetime import datetime

# cpu usage
cpu_usage = psutil.cpu_percent(interval=1)  # CPU usage in percentage.
print(f"cpu_usage", cpu_usage)

# memory
memory_info = psutil.virtual_memory()
memory_usage = {
    "total": f"{memory_info.total / (1024 ** 3):.2f} GB",
    "used": f"{memory_info.used / (1024 ** 3):.2f} GB",
    "free": f"{memory_info.free / (1024 ** 3):.2f} GB",
}
print(f"memory_usage", memory_usage)

#uptime
uptime = datetime.now() - datetime.fromtimestamp(psutil.boot_time())  # System uptime.
print(f"uptime", uptime)


#os info
os_info = {
    "os_name": platform.system(),
    "os_version": platform.version(),
    "kernel_version": platform.release(),
}
print(f"os_info: {os_info}")


# timezone based time
# e.g., 'America/New_York'
try:
    current_time = datetime.now(ZoneInfo(timezone_name))
    # return current_time.strftime("%Y-%m-%d %H:%M:%S %Z")
except Exception as e:
    pass
    # return f"Error: Invalid timezone: {str(e)}"

cpu_usage 50.5
memory_usage {'total': '15.34 GB', 'used': '2.39 GB', 'free': '1.88 GB'}
uptime 0:04:53.732463
os_info: {'os_name': 'Linux', 'os_version': '#77~20.04.1-Ubuntu SMP Thu Oct 3 19:39:59 UTC 2024', 'kernel_version': '5.15.0-1071-aws'}


In [155]:
#think of anything , do chatgpt and try to implement

import speedtest

@tool(parse_docstring=True)
def get_wifi_speed():
    """Retrieves wifi speed. Call this whenever you need to know the wifi speed, for example when a customer asks "What is the wifi speed?" or "How fast is internet?"
    Args: None

    Returns:
        dict: A dictionary containing wifi speed statistics with the following keys:
            - download_speed_mbps (str): Total download speed in Mbps
            - upload_speed_mbps (str): Total upload speed in Mbps
    """
    # Initialize the Speedtest client
    st = speedtest.Speedtest()
    
    # Fetch the best server
    st.get_best_server()
    
    # Measure download and upload speeds
    download_speed = st.download() / (1024 * 1024) 
    upload_speed = st.upload() / (1024 * 1024) 
    
    # Return results
    return {
        "download_speed_mbps": f"{download_speed:.2f} Mbps",
        "upload_speed_mbps": f"{upload_speed:.2f} Mbps"
    }


In [156]:

tools_list = {
    "get_disk_usage": get_disk_usage,
    "get_wifi_speed": get_wifi_speed,
    
}

# Initialize LLM
llm = ChatOllama(model="qwen2.5-coder:7b")
llm_with_tools = llm.bind_tools(list(tools_list.values()))

In [181]:
messages =[]
prompt = "How is the internet today? "
messages.append(HumanMessage(prompt))
ai_response = llm_with_tools.invoke(messages)
ai_response.tool_calls

[{'name': 'get_wifi_speed',
  'args': {},
  'id': '7dcb7c30-522e-4682-a922-8c2156424873',
  'type': 'tool_call'}]

In [183]:
messages = [] 
prompt = "I want to download 500 GB of data from Youtube, can I download it within 2 hour ?"
messages.append(HumanMessage(prompt))
ai_response = llm_with_tools.invoke(messages)
ai_response.tool_calls

[{'name': 'get_wifi_speed',
  'args': {},
  'id': 'df2410ac-afdc-4d43-a736-9cd262b0ebf1',
  'type': 'tool_call'}]

In [184]:
messages = [] 
prompt = "I want to download 500 GB of data from Youtube, can I download it in small machine ?"
messages.append(HumanMessage(prompt))
ai_response = llm_with_tools.invoke(messages)
ai_response.tool_calls

[{'name': 'get_wifi_speed',
  'args': {},
  'id': 'ca953f0b-b077-4106-bf85-d576dc6ea69e',
  'type': 'tool_call'},
 {'name': 'get_disk_usage',
  'args': {},
  'id': 'ad84925a-cd19-453a-9659-77a771e4f344',
  'type': 'tool_call'}]

In [185]:
messages = [] 
prompt = "I want to download 500 GB of data from Youtube, can I download it in my machine without any time limit?"
messages.append(HumanMessage(prompt))
ai_response = llm_with_tools.invoke(messages)
ai_response.tool_calls

[{'name': 'get_disk_usage',
  'args': {},
  'id': '35bde50a-33f4-4dbd-afb8-e01a9eaf9e46',
  'type': 'tool_call'},
 {'name': 'get_wifi_speed',
  'args': {},
  'id': '8fb40cdf-0e82-4d45-a331-f0fd568fdb86',
  'type': 'tool_call'}]

In [186]:
# Handle AI response
if not ai_response.tool_calls:
    print("\nAI Response:")
    print(ai_response.content)

# Execute tool calls if any
for tool_call in ai_response.tool_calls:
    selected_tool = tools_list.get(tool_call["name"].lower())
    if selected_tool:
        print(selected_tool, tool_call)
        tool_response = selected_tool.invoke(tool_call["args"])
        messages.append(ToolMessage(tool_response, tool_call_id=tool_call["id"]))
    else:
        print(f"Error: Tool '{tool_call['name']}' not found.")

name='get_disk_usage' description='Retrieves disk usage. Call this whenever you need to know the disk usage, for example when a customer asks "What is the disk usage?"\nArgs: None' args_schema=<class 'langchain_core.utils.pydantic.get_disk_usage'> func=<function get_disk_usage at 0x7f2ba3a07b50> {'name': 'get_disk_usage', 'args': {}, 'id': '35bde50a-33f4-4dbd-afb8-e01a9eaf9e46', 'type': 'tool_call'}
name='get_wifi_speed' description='Retrieves wifi speed. Call this whenever you need to know the wifi speed, for example when a customer asks "What is the wifi speed?" or "How fast is internet?"\nArgs: None' args_schema=<class 'langchain_core.utils.pydantic.get_wifi_speed'> func=<function get_wifi_speed at 0x7f2ba3a05870> {'name': 'get_wifi_speed', 'args': {}, 'id': '8fb40cdf-0e82-4d45-a331-f0fd568fdb86', 'type': 'tool_call'}


In [188]:
# Stream the final response
print("\nFinal AI Response:")
final_response = llm_with_tools.stream(messages)
for part in final_response:
    print(part, end="")
print("\n")


Final AI Response:
content='<response>\n    "message": "Based on the provided information, you have 471.09 GB of free disk space and a download speed of 2.57 Mbps. Downloading 500 GB of data might exceed your available disk space, but it depends on your internet speed. If your download speed is faster than your upload speed, you should be able to complete the download without any time limit. However, if your download speed is slower than your upload speed, it could take longer or may not even be possible to start the download due to network limitations."\n</response>' additional_kwargs={} response_metadata={'model': 'qwen2.5-coder:7b', 'created_at': '2024-11-16T23:29:25.344353792Z', 'message': {'role': 'assistant', 'content': '<response>\n    "message": "Based on the provided information, you have 471.09 GB of free disk space and a download speed of 2.57 Mbps. Downloading 500 GB of data might exceed your available disk space, but it depends on your internet speed. If your download spe

In [197]:
part.content

'<response>\n    "message": "Based on the provided information, you have 471.09 GB of free disk space and a download speed of 2.57 Mbps. Downloading 500 GB of data might exceed your available disk space, but it depends on your internet speed. If your download speed is faster than your upload speed, you should be able to complete the download without any time limit. However, if your download speed is slower than your upload speed, it could take longer or may not even be possible to start the download due to network limitations."\n</response>'